# Community alignment of training questions

## Imports

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

## Question sample

In [ ]:
from forecasting_tools import MetaculusApi, ApiFilter
from datetime import datetime, timedelta
import glob, joblib, os

In [ ]:
from load_forecasted_open_questions import load_forecasted_open_questions

In [ ]:
num_questions = (0,100000)

In [ ]:
questions = load_forecasted_open_questions(num_questions)

In [ ]:
questions_binary = [question for question in questions if question.api_json['question']['type'] == 'binary']

In [ ]:
questions_multiple_choice = [question for question in questions if question.api_json['question']['type'] == 'multiple_choice']

In [ ]:
questions_numeric = [question for question in questions if question.api_json['question']['type'] == 'numeric']

In [ ]:
[len(x) for x in [questions_binary, questions_multiple_choice, questions_numeric]]

## Community forecast

In [ ]:
from community_forecast import *

In [ ]:
id_to_forecast = {question.api_json['id']: community_forecast(question) for question in questions}

In [ ]:
id_to_question = {question.api_json['id']: question for question in questions}

In [ ]:
community_forecast(questions_binary[0])

In [ ]:
community_forecast(questions_multiple_choice[0])

In [ ]:
questions_multiple_choice[0].api_json['id']

In [ ]:
community_forecast(questions_numeric[0])

## 000 forecast

In [ ]:
import glob
fns = glob.glob('forecast_community/*.md')

In [ ]:
type(questions_numeric[0])

In [ ]:
fn = fns[0]

In [ ]:
ids = [int(fn.split('/')[1].split('.')[0]) for fn in fns]

In [ ]:
forecasts = {id: open(f'forecast_community/{id}.md').read() for id in ids}

In [ ]:
community_ids = list(id_to_forecast.keys())

In [ ]:
type(community_ids[0])

In [ ]:
forecast_ids = list(forecasts.keys())

In [ ]:
done = list(set(forecast_ids).intersection(community_ids))

In [ ]:
id = 26327
question = id_to_question[id]

In [ ]:
forecasts[id]

In [ ]:
from extract_forecast import *

def extract_only_forecast(id):
    question = id_to_question[id]
    question_type = type(question)
    forecast = forecasts[id]
    if question_type == forecasting_tools.data_models.questions.BinaryQuestion:
        prediction = extract_probability_from_response_as_percentage_not_decimal(forecast)/100.0
    elif question_type == forecasting_tools.data_models.questions.MultipleChoiceQuestion:
        options = question.options
        option_probabilities = extract_option_probabilities_from_response(forecast, options)
        prediction = generate_multiple_choice_forecast(options, option_probabilities)
    elif question_type == forecasting_tools.data_models.questions.NumericQuestion:
        prediction = extract_percentiles_from_response(forecast)
    return prediction

In [ ]:
import forecasting_tools
extract_only_forecast(id)

In [ ]:
for id in done:
    print(id)
    print(extract_only_forecast(id))

In [ ]:
predictions = {id: extract_only_forecast(id, txt) for id,txt in forecasts.items()}

## Forecast missing questions

In [ ]:
missing = list(sorted(set(community_ids).difference(forecast_ids)))

In [ ]:
len(missing)

In [ ]:
missing[0]

In [ ]:
from forecast import forecast

In [ ]:
num_questions = (missing[0], missing[0]+1)
perennial = False
live = False

In [ ]:
results = forecast(num_questions = num_questions, perennial = perennial, live=False)